In [25]:
import pickle
from collections import defaultdict
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import os
import numpy as np
import operator
import pandas as pd
from collections import defaultdict, Counter
from operator import itemgetter
import nltk

In [26]:
corpusdir = "/home/dhaval/Acads/Spring19/Information Retrieval/Project/Parsed_Files/"

In [27]:
newcorpus = PlaintextCorpusReader(corpusdir, '.*')

In [28]:
def get_Tokens(filename):
    file_content = open(corpusdir + filename).read()
    tokens = nltk.word_tokenize(file_content)
    return tokens

In [29]:
with open('unigrams.pickle','rb') as f:
    unigrams = pickle.load(f)

In [30]:
with open('parsed_queries.pickle','rb') as f:
    queries = pickle.load(f)

In [31]:
with open('relevance.pickle','rb') as f:
    rel = pickle.load(f)

In [32]:
with open('common_words') as f:
    content = f.read()
    stop_list = nltk.word_tokenize(content)

In [48]:
def removeStopWords(l):
    k = []
    for word in l:
        if word not in stop_list and not(word.isnumeric()):
            k.append(word)
    return k

In [33]:
relevance = {}

In [34]:
for key,value in rel.items():
    val = []
    for v in value:
        val.append(v+".txt")
    relevance[int(key)] = val

In [35]:
tot_length = 0

In [36]:
for filename in newcorpus.fileids():
    count = len(open(corpusdir+filename).readlines(  ))
    tot_length+=count

In [37]:
avdl = tot_length/3204
k1=1.2
b=0.75
k2=100
N = 3204

In [38]:
def getr(term,qid):
    list_of_docs = list(map(itemgetter(0), unigrams[term]))
    relevant_docs = relevance[qid]
    r = list(set(list_of_docs).intersection(set(relevant_docs)))
    return len(r)

In [49]:
def getRanking(qid,query,t):
    try:
        R = len(relevance[qid])
    except:
        R = 0
    terms = query.split()
    query_id = qid
    frequency = defaultdict(int)
    for word in removeStopWords(terms):
        frequency[word]+=1
    Score = defaultdict(float)
    for term in frequency.keys():
        l = unigrams[term]
        qf = frequency[term]
        if(R!=0):
            r = getr(term,qid)
        else:
            r = 0
        n = len(l)
        for doc in l:
            dl = len(open(corpusdir+doc[0]).readlines())
            K = k1*((1-b)+b*dl/avdl)
            f = doc[1]
            d = (r+0.5)*(N-n-R+r+0.5)/((R-r+0.5)*(n-r+0.5))
            BM25 = np.log(d)*(k1+1)*f*(k2+1)*qf/((K+f)*(k2+qf))
            Score[doc[0]]+=BM25
    sorted_Score = sorted(Score.items(), key=operator.itemgetter(1), reverse = True)
    column_names = ['doc_id','BM25_score']
    df = pd.DataFrame(sorted_Score[0:t],columns = column_names)
    df.insert(1, "rank", np.arange(len(df))+1, True)
    df['system_name'] = "BM25"
    df.insert(0, "Q0", "Q0", True)
    df.insert(0, "query_id", query_id, True)
    return df
    #df.to_csv("query_"+str(query_id)+"_rankings_.csv",index=False)

In [40]:
top_docs = {}
for key,value in queries.items():
    df = list(getRanking(key,value,10)['doc_id'])
    top_docs[key] = df

In [42]:
def top_words(most_occur,words_in_query,k):
    i = 0
    l = []
    for word in most_occur:
        if(word[0] not in words_in_query.split()):
            i = i+1
            l.append(word[0])
        if i==k:
            break
    return l

In [43]:
most_common = {}
for key,value in top_docs.items():
    tokens = []
    for doc in value:
        tokens = tokens + (get_Tokens(doc))
    tokens = removeStopWords(tokens)
    counter = Counter(tokens)
    most_occur = counter.most_common(50)
    words_in_query = queries[key]
    most_common[key] = top_words(most_occur,words_in_query,5)    

In [44]:
most_common

{1: ['cacm', 'jb', 'program', 'time-sharing', 'computer'],
 2: ['computer', 'system', 'cacm', 'systems', 'page'],
 3: ['computer', 'system', 'cacm', 'number', 'science'],
 4: ['system', 'cacm', 'computer', 'language', 'information'],
 5: ['system', 'language', 'retrieval', 'data', 'graphic'],
 6: ['computer', 'cacm', 'jb', 'simulation', 'digital'],
 7: ['routing', 'network', 'cacm', 'problem', 'systems'],
 8: ['computer', 'system', 'cacm', 'information', 'jb'],
 9: ['information', 'model', 'system', 'cacm', 'data'],
 10: ['io', 'algorithm', 'cacm', 'pm', 'jb'],
 11: ['programming', 'language', 'data', 'cacm', 'systems'],
 12: ['system', 'random', 'cacm', 'security', 'language'],
 13: ['expressions', 'cacm', 'program', '4.12', 'jb'],
 14: ['cacm', 'jb', 'algorithm', 'march', 'pm'],
 15: ['algorithm', 'analysis', 'program', 'cacm', 'flow'],
 16: ['system', 'process', 'cacm', 'set', 'jb'],
 17: ['cacm', '4.12', 'jb', 'pm', 'compiler'],
 18: ['algorithm', 'time', 'io', 'cacm', 'processing'

In [45]:
queries_expanded = {}
for key, value in queries.items():
    l = value.split()
    l = l + most_common[key]
    queries_expanded[key] = ' '.join(l)

In [46]:
queries_expanded

{1: 'what articles exist which deal with tss time sharing system an operating system for ibm computers cacm jb program time-sharing computer',
 2: 'i am interested in articles written either by prieve or udo pooch prieve b pooch u computer system cacm systems page',
 3: 'intermediate languages used in construction of multitargeted compilers tcoll computer system cacm number science',
 4: 'im interested in mechanisms for communicating between disjoint processes possibly but not exclusively in a distributed environment i would rather see descriptions of complete mechanisms with or without implementations as opposed to theoretical work on the abstract problem remote procedure calls and messagepassing are examples of my interests system cacm computer language information',
 5: 'id like papers on design and implementation of editing interfaces windowmanagers command interpreters etc the essential issues are human interface design with views on improvements to user efficiency effectiveness a

In [50]:
for key,value in queries_expanded.items():
    df = getRanking(key,value,50)
    df.to_csv("Query_expansion_results/query_"+str(key)+"_rankings_.csv",index=False)

/home/dhaval/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in log
